<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Copia_de_Modelo_de_Mezcla_de_Agentes_con_Modelo_Oculto_de.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¡Hola\! Sí, esta propuesta es significativamente más compleja que la anterior, pero es una idea excelente y muy relevante en la neurociencia computacional y el aprendizaje por refuerzo. Describe un modelo de vanguardia para entender la flexibilidad cognitiva.

Tienes toda la razón, para implementarlo es mejor empezar con una versión simplificada que capture la esencia del modelo. Aplaudo tu intuición de proponer la simplificación.

Aquí te presento un plan que reduce la complejidad inicial pero mantiene el propósito central de tu simulador: ver cómo un agente cambia de "estrategia" (estado oculto) a lo largo del tiempo.

-----

### **Plan de Simplificación**

Para hacer este simulador manejable y educativo, nos centraremos en los siguientes puntos:

1.  **Reducir el Número de Agentes**: En lugar de los cinco agentes, nos enfocaremos en el conflicto más clásico e informativo:

      * **Agente Model-Free (MFr)**: Aprende valores simples por Q-learning. Es rápido pero inflexible.
      * **Agente Model-Based (MBr)**: Actúa como si conociera la estructura de la tarea para planificar. Es "inteligente" pero lento.
      * **Agente de Sesgo (Bias)**: Una simple preferencia por una opción, para capturar tendencias generales.

2.  **Simplificar la Tarea del Agente**: En lugar de una tarea compleja (como la de dos etapas o "two-step task") que requiere que el agente MBr aprenda un modelo del mundo, simularemos una tarea más simple. Podemos "emular" el comportamiento del agente MBr dándole conocimiento a priori sobre la mejor acción, sin necesidad de simular su proceso de planificación explícitamente.

3.  **Enfocarse en la Simulación (Generación) y no en la Inferencia**: El problema se puede dividir en dos:

      * **Simulación (Generativo)**: Usar el HMM para *generar* una secuencia de estados ocultos y, a partir de ellos, generar elecciones. **Empezaremos aquí.** Es más sencillo y nos permite saber cuál es el estado "real" en cada momento.
      * **Análisis (Inferencia)**: Usar las elecciones observadas para *inferir* cuál fue la secuencia de estados más probable. Este es un paso más avanzado (requiere algoritmos como el de Viterbi o Forward-Backward) que podemos dejar para una segunda versión.

4.  **Fijar Algunos Parámetros del HMM**: En lugar de tener sliders para cada una de las probabilidades en la matriz de transición, podemos empezar con una matriz predefinida o con sliders que controlen parámetros más simples (ej. "probabilidad de permanecer en el mismo estado").

-----

### **Lógica del Modelo Simplificado (Ensayo por Ensayo)**

Para cada ensayo `t`, el simulador seguirá estos pasos:

1.  **Transición del Estado Oculto**: Primero, el HMM determina el estado oculto del ensayo actual, `S_t`. Pasa del estado anterior `S_{t-1}` al nuevo estado `S_t` según las probabilidades de la **Matriz de Transición (P)**. Por ejemplo, si estaba en el estado 1 ("Modo Racional"), podría pasar al estado 2 ("Modo Hábito") con una probabilidad `P(S_t=2 | S_{t-1}=1)`.

2.  **Cálculo del Valor de Cada Agente**: Cada uno de los tres agentes (MFr, MBr, Bias) calcula el valor que le asigna a cada elección posible (ej. izquierda o derecha).

      * `V_MFr`: Valores basados en el historial de recompensas pasadas (Q-learning).
      * `V_MBr`: Valores basados en el conocimiento de la "mejor" acción en ese momento.
      * `V_Bias`: Valores constantes que reflejan una preferencia (ej. `[0.1, -0.1]` para un sesgo a la izquierda).

3.  **Combinación de Agentes (la "Mezcla")**: Aquí está la magia. Dependiendo del estado oculto actual `S_t`, los valores de los agentes se combinan usando los **pesos (β)** específicos de ese estado.

      * `Valor_Total = β_MFr(S_t) * V_MFr + β_MBr(S_t) * V_MBr + β_Bias(S_t) * V_Bias`
      * Por ejemplo, si `S_t` es el "Estado Racional", los pesos podrían ser `β_MBr=5, β_MFr=1, β_Bias=0.5`. Si es el "Estado Hábito", `β_MBr=1, β_MFr=5, β_Bias=0.5`.

4.  **Toma de Decisión**: El `Valor_Total` se pasa por una función **softmax** para generar una probabilidad de elección. El agente elige una acción (izquierda/derecha) basándose en esa probabilidad.

5.  **Actualización del Aprendizaje**: Una vez que se recibe la recompensa, el agente **Model-Free (MFr)** actualiza sus valores `V_MFr` para el siguiente ensayo. Los otros agentes en nuestro modelo simplificado no necesitan actualizarse.

-----

### **Estructura del Código Python**

Para implementar esto, propongo una estructura con clases que separen las responsabilidades:

In [ ]:
# Clases para cada agente individual
class AgentMFr:
    def __init__(self, learning_rate):
        # ...
    def get_values(self):
        # Retorna sus valores Q para las elecciones
        pass
    def update(self, choice, reward):
        # Actualiza sus valores Q
        pass

class AgentMBr:
    def get_values(self, task_structure):
        # Retorna el valor 'óptimo' basado en la estructura de la tarea
        pass

# Clase principal del simulador
class MoA_HMM_Simulator:
    def __init__(self, agents, hmm_params):
        # Almacena los agentes (MFr, MBr, Bias)
        # Almacena los parámetros del HMM:
        #   - Pesos beta para cada estado
        #   - Matriz de transición P
        #   - Probabilidad inicial pi
        self.agents = agents
        self.hmm_params = hmm_params
        self.current_hidden_state = ...

    def _get_next_hidden_state(self):
        # Usa la matriz de transición para pasar al siguiente estado
        pass

    def run_single_trial(self):
        # Implementa los 5 pasos de la lógica descrita arriba
        # 1. Obtener nuevo estado oculto
        # 2. Obtener valores de cada agente
        # 3. Combinar valores con los pesos beta del estado actual
        # 4. Tomar una decisión (softmax)
        # 5. Actualizar los agentes que aprenden (MFr)
        pass

    def run_full_simulation(self, num_trials):
        # Llama a run_single_trial en un bucle
        # y guarda el historial para graficar
        pass

Este enfoque simplificado nos permite tener un simulador funcional rápidamente, que sigue siendo extremadamente potente para visualizar cómo las estrategias pueden cambiar de forma dinámica y latente a lo largo del tiempo.

**¿Te parece bien este plan de simplificación para empezar?** Si estás de acuerdo, puedo proceder a escribir el código de esta primera versión.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, Layout
from IPython.display import display

# --- 1. Definición de los Agentes Individuales ---

class AgentMFr:
    """Agente Model-Free (MFr) que usa Q-learning."""
    def __init__(self, learning_rate=0.2, num_actions=2):
        self.alpha = learning_rate
        self.q_values = np.zeros(num_actions)

    def get_values(self):
        return self.q_values.copy()

    def update(self, choice, reward):
        prediction_error = reward - self.q_values[choice]
        self.q_values[choice] += self.alpha * prediction_error

class AgentMBr:
    """Agente Model-Based (MBr) 'idealizado' que conoce la mejor acción."""
    def get_values(self, best_action):
        # Devuelve un valor alto para la mejor acción conocida y bajo para la otra.
        values = np.full(2, 0.1)
        values[best_action] = 0.9
        return values

class AgentBias:
    """Agente de Sesgo que prefiere una acción sobre la otra."""
    def __init__(self, bias_direction=0, bias_strength=0.1):
        self.values = np.zeros(2)
        self.values[bias_direction] = bias_strength

    def get_values(self):
        return self.values

# --- 2. Definición del Entorno de la Tarea ---

class TaskEnvironment:
    """Gestiona la tarea de reversión probabilística."""
    def __init__(self, num_trials=200, reversal_point=100):
        self.num_trials = num_trials
        self.reversal_point = reversal_point
        # Probabilidades iniciales: la acción 0 es mejor
        self.reward_probs = [0.8, 0.2]
        self.current_trial = 0

    def get_reward(self, choice):
        self.current_trial += 1
        if self.current_trial == self.reversal_point:
            # Se invierten las probabilidades
            self.reward_probs.reverse()

        # Devuelve una recompensa basada en la probabilidad de la elección
        return 1 if np.random.rand() < self.reward_probs[choice] else 0

    def get_best_action(self):
        return np.argmax(self.reward_probs)

# --- 3. El Simulador Principal MoA-HMM ---

def softmax(x, temp=1.0):
    """Calcula probabilidades usando la función softmax."""
    e_x = np.exp((x - np.max(x)) / temp)
    return e_x / e_x.sum(axis=0)

class MoA_HMM_Simulator:
    def __init__(self, hmm_params, agents):
        self.P = hmm_params['P']          # Matriz de transición de estados
        self.pi = hmm_params['pi']        # Probabilidad inicial de estado
        self.betas = hmm_params['betas']  # Pesos de los agentes por estado
        self.temp = hmm_params['temp']    # Temperatura del softmax

        self.agents = agents
        self.num_states = self.P.shape[0]
        self.history = {}

    def run_simulation(self, num_trials=200):
        # Inicializar
        task = TaskEnvironment(num_trials)
        [agent.q_values.fill(0) for agent in self.agents if isinstance(agent, AgentMFr)]
        self.history = {'hidden_state': [], 'choice': [], 'reward': [], 'agent_weights': []}

        # Estado inicial del HMM
        current_state = np.random.choice(self.num_states, p=self.pi)

        for t in range(num_trials):
            self.history['hidden_state'].append(current_state)

            # 1. Obtener valores de cada agente
            v_mfr = self.agents[0].get_values()
            v_mbr = self.agents[1].get_values(task.get_best_action())
            v_bias = self.agents[2].get_values()

            # 2. Combinar valores según el estado oculto actual
            current_betas = self.betas[current_state]
            v_total = current_betas['mfr'] * v_mfr + \
                      current_betas['mbr'] * v_mbr + \
                      current_betas['bias'] * v_bias

            self.history['agent_weights'].append([current_betas['mfr'], current_betas['mbr']])

            # 3. Tomar una decisión
            choice_probs = softmax(v_total, self.temp)
            choice = np.random.choice(len(v_total), p=choice_probs)
            self.history['choice'].append(choice)

            # 4. Obtener recompensa y actualizar agentes
            reward = task.get_reward(choice)
            self.history['reward'].append(reward)
            self.agents[0].update(choice, reward) # Solo el MFr aprende

            # 5. Transición al siguiente estado oculto
            current_state = np.random.choice(self.num_states, p=self.P[current_state])

        return self.history

# --- 4. Función para Configurar y Correr la Simulación Interactiva ---

def run_and_plot_simulation(p_stay_0, p_stay_1, temp, alpha_mfr):

    # Parámetros fijos de los estados ocultos
    # Estado 0: "Racional" (dominado por MBr)
    # Estado 1: "Hábito" (dominado por MFr)
    betas = {
        0: {'mbr': 5.0, 'mfr': 1.0, 'bias': 0.1},
        1: {'mbr': 1.0, 'mfr': 5.0, 'bias': 0.1}
    }

    # Construir la matriz de transición desde los sliders
    P = np.array([
        [p_stay_0, 1 - p_stay_0],  # Probs de transición desde el estado 0
        [1 - p_stay_1, p_stay_1]   # Probs de transición desde el estado 1
    ])

    hmm_params = {
        'P': P,
        'pi': [0.5, 0.5], # Probabilidad inicial de empezar en cada estado
        'betas': betas,
        'temp': temp      # "Ruido" en la decisión
    }

    agents = [AgentMFr(learning_rate=alpha_mfr), AgentMBr(), AgentBias()]

    simulator = MoA_HMM_Simulator(hmm_params, agents)
    history = simulator.run_simulation(num_trials=200)

    # --- Graficar los resultados ---
    fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
    trials = np.arange(1, 201)

    # Gráfico 1: Trayectoria del Estado Oculto
    ax1 = axes[0]
    ax1.fill_between(trials, 0, 1, where=np.array(history['hidden_state']) == 0, color='skyblue', alpha=0.6, label='Estado 0 (Racional/MBr)')
    ax1.fill_between(trials, 0, 1, where=np.array(history['hidden_state']) == 1, color='salmon', alpha=0.6, label='Estado 1 (Hábito/MFr)')
    ax1.set_yticks([])
    ax1.set_title('1. Trayectoria del Estado Oculto Generado por el HMM')
    ax1.legend(loc='upper center', ncol=2)
    ax1.axvline(100.5, color='k', linestyle=':', lw=2, label='Inversión de Recompensas')

    # Gráfico 2: Contribución Ponderada de Agentes
    ax2 = axes[1]
    weights = np.array(history['agent_weights']).T
    ax2.stackplot(trials, weights[1], weights[0], labels=['Model-Based (MBr)', 'Model-Free (MFr)'], colors=['skyblue', 'salmon'])
    ax2.set_title('2. Influencia de los Agentes en la Decisión')
    ax2.set_ylabel('Peso (β) en la Decisión')
    ax2.legend(loc='upper center', ncol=2)
    ax2.axvline(100.5, color='k', linestyle=':', lw=2)

    # Gráfico 3: Elecciones y Resultados
    ax3 = axes[2]
    choices = np.array(history['choice'])
    rewards = np.array(history['reward'])
    ax3.scatter(trials[choices==0], choices[choices==0] + 0.05, marker='|', color='blue', label='Elección 0', alpha=0.7)
    ax3.scatter(trials[choices==1], choices[choices==1] - 0.05, marker='|', color='green', label='Elección 1', alpha=0.7)
    ax3.scatter(trials[rewards==1], rewards[rewards==1] - choices[rewards==1], marker='o', facecolors='none', edgecolors='gold', s=80, label='Recompensa')
    ax3.set_yticks([0, 1])
    ax3.set_ylabel('Elección')
    ax3.set_xlabel('Ensayo')
    ax3.set_title('3. Comportamiento del Agente')
    ax3.legend(loc='center right')
    ax3.axvline(100.5, color='k', linestyle=':', lw=2)

    plt.tight_layout()
    plt.show()

# --- Crear los Widgets Interactivos ---
style = {'description_width': 'initial'}
layout = Layout(width='450px')
interactive_plot = interactive(
    run_and_plot_simulation,
    p_stay_0=FloatSlider(value=0.95, min=0.5, max=1.0, step=0.01, description='P(Quedarse en Estado 0 "Racional"):', style=style, layout=layout),
    p_stay_1=FloatSlider(value=0.95, min=0.5, max=1.0, step=0.01, description='P(Quedarse en Estado 1 "Hábito"):', style=style, layout=layout),
    temp=FloatSlider(value=0.2, min=0.05, max=1.0, step=0.05, description='Temperatura Softmax (Ruido):', style=style, layout=layout),
    alpha_mfr=FloatSlider(value=0.2, min=0.05, max=1.0, step=0.05, description='Tasa de Aprendizaje MFr (α):', style=style, layout=layout)
)
display(interactive_plot)

interactive(children=(FloatSlider(value=0.95, description='P(Quedarse en Estado 0 "Racional"):', layout=Layout…

Cómo Interpretar los Resultados
Gráfico 1 (Superior): Muestra el estado oculto "real" en el que se encuentra el agente en cada ensayo. El azul es el "Modo Racional" (domina MBr) y el rojo es el "Modo Hábito" (domina MFr). Observa cómo el agente cambia entre estos modos según las probabilidades de transición que definiste.

Gráfico 2 (Medio): Visualiza qué agente tiene más influencia en cada momento. Verás que el área azul es mayor cuando el estado oculto es 0, y el área roja es mayor cuando el estado oculto es 1. Esto muestra la "mezcla" de agentes en acción.

Gráfico 3 (Inferior): Muestra el comportamiento observable. Las líneas verticales son las elecciones y los círculos dorados son las recompensas. Fíjate qué pasa después de la inversión de recompensas (línea negra punteada):

Si el agente está en el modo Racional (azul), debería adaptarse rápidamente al cambio, ya que el agente MBr sabe cuál es la nueva mejor opción.

Si está en el modo Hábito (rojo), seguirá eligiendo la opción que antes era buena, porque el agente MFr es lento para reaprender. Cometerá muchos errores hasta que su valor Q se actualic